<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://blog.scholarnest.com/wp-content/uploads/2023/03/scholarnest-academy-scaled.jpg" alt="ScholarNest Academy" style="width: 1400px">
</div>

###Creating your DBFS mount for data storage
1. Create ADLS Gen2 Storage account
2. Create storage container in your storage account
3. Create Azure service principal and secret
4. Grant access to service proncipal for storage account
5. Mount storage container

####1. Create ADLS Gen2 Storage account
* Click "Create a resource" on your Azure portal home page
* Search for "Storage account" and click the create button
* Create a storage account using the following
    * Choose an appropriate subscription
    * Select an existing or Create a new Resource group
    * Choose a unique storage account name
    * Choose a region (Choose the same region where your Databricks service is created)
    * Select performance tier (Standard tier is good enough for learning)
    * Choose storage redundency (LRS is good enough for learning)
    * Click Advanced button to move to the next step
    * Select "Enable hierarchical namespace" on the Advanced tab
    * Click "Review" button
    * Click the "Create" button after reviewing your settings


1. Go to azure portal and click on a + to "create a new resource"
2. Find "storage account" and create new storage:
  * choose the subscription and resource group
  * give storage account name (must be unique across all azure globally!!!!)
  * choose region (should be the same like region in workspace)
  * performance - choose standard or premium
  * choose redudnancy storage - for use is locally for the lerning
3. Go to tab addvence and go to "Enable hierarchical namespace" and choose these (it is complemented by Data Lake Storgae Gen2)
4. Create "REVIEW" (we do not set more in thie excersise)

![obraz_1769947013197.png](./obraz_1769947013197.png "obraz_1769947013197.png")

####2. Create storage container in your storage account
* Go to your Azure storage account page
* Select "Containers" from the left side menu
* Click "+ Container" button from the top menu
* Give a name to your containe (Ex dbfs-container)
* Click the "Create" button

![obraz_1769947152557.png](./obraz_1769947152557.png "obraz_1769947152557.png")

####3. Create Azure service principal and secret
* Go to Azure Active Directory Service page in your Azure account (Azure Active Directory is now Microsoft Entra ID)
* Select "App registrations" from the left side menu - WE WANT TO CREATE A SERVICE ACCOUNT
* Click (+ New registration) from the top menu
* Give a name to your service principal (Ex databricks-app-principal)
* Click the "Register" button
* Service principal will be created and details will be shown on the service principal page

![obraz_1769947405230.png](./obraz_1769947405230.png "obraz_1769947405230.png")

* Copy "Application (client) ID" and "Directory (tenant) ID" values. You will need them later
* Choose "Certificates & secrets" from the left menu
* Click "+ New client secret" on the secrets page
* Enter a description (Ex databricks-app-principal-secret)
* Select an expiry (Ex 3 Months)
* Click the "Add" button
* Secret will be created and shown on the page
* Copy the Secret value. You will need it later - IT IS VISIBLE ONLY ONCE!!! 

![obraz_1769949085530.png](./obraz_1769949085530.png "obraz_1769949085530.png")



####4. Grant access to service principal for storage account
* Go to your storage account page
* Click "Access control (IAM)" from the left menu
* Click the "+ Add" button and choose "Add role assignment"
* Search for "Storage Blob Data Contributor" role and select it
* Click "Next" button
* Click the "+ Select members"
* Search for your Databricks service principal (Ex databricks-app-principal) and select it
* Clcik "Select" button
* Click "Review + assign" button twice
![obraz_1769949408650.png](./obraz_1769949408650.png "obraz_1769949408650.png")

####5. Mount storage container

#####5.1 Define necessory variables

In [0]:
#very bad solution, the data have been anomised
storage_account_name = "XXXXX"
container_name = "XXXXXX"
mount_point = "XXXXXXX"
client_id = "XXXXXXXX"
tenent_id = "XXXXXXXXXX"
client_secret = "XXXXXXXXXX"

#####5.2 Define mount configs
You can follow the instruction and code sample from below documentation page

https://learn.microsoft.com/en-us/azure/databricks/dbfs/mounts#--mount-adls-gen2-or-blob-storage-with-abfs

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id, #"<application-id>"
          "fs.azure.account.oauth2.client.secret": client_secret, 
          # dbutils.secrets.get(scope="<scope-name>",key="<service-credential-key-name>") - not used here
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenent_id}/oauth2/token"
          }


#####5.3 Mount the container

In [0]:
# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
  mount_point = f"/mnt/{mount_point}"
  ,  extra_configs = configs
  )

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-34647130514052>, line 2
      1 # Optionally, you can add <directory-name> to the source URI of your mount point.
----> 2 dbutils.fs.mount(
      3   source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
      4   mount_point = f"/mnt/{mount_point}"
      5   ,  extra_configs = configs
      6   )

File /databricks/python_shell/lib/dbruntime/dbutils.py:172, in prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    170 exc.__context__ = None
    171 exc.__cause__ = None
--> 172 raise exc

ExecutionError: An error occurred while calling o478.mount.
: com.databricks.rpc.FeatureDisabledException: DBFS mounts are not available on this workspace
	at com.databricks.backend.daemon.data.client.BaseDbfsClient.send0(BaseDbfsClient.scala:191)
	at com.databricks.backend.daemon.da

In [0]:
print(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/")

abfss://XXXXXX@XXXXX.dfs.core.windows.net/


#####5.4. List contents of your mount point

In [0]:
%fs ls /mnt/files-data

#####5.5. Upload files to your mounted location

#####5.6. List contents of your mount point

In [0]:
%fs dbfs

#####5.7. Unmount /mnt/data directory

&copy; 2021-2023 ScholarNest Technologies Pvt. Ltd. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
Databricks, Databricks Cloud and the Databricks logo are trademarks of the <a href="https://www.databricks.com/">Databricks Inc</a>.<br/>
<br/>
<a href="https://www.scholarnest.com/privacy/">Privacy Policy</a> | 
<a href="https://www.scholarnest.com/terms/">Terms of Use</a> | <a href="https://www.scholarnest.com/contact/">Contact Us</a>